# HW7-Part1_Myunghee_ID_2446752777

# 1. Generative Models for Text

(c) LSTM: Train an LSTM to mimic Russell’s style and thoughts:

i. Concatenate your text ﬁles to create a corpus of Russell’s writings. 

In [1]:
file1="./books/TPP.txt"  # The Problems of Philosophy
file2="./books/TAM.txt"  # The Analysis of Mind
file3="./books/MLOE.txt" # Mysticism and Logic and Other Essays
file4="./books/OKEWFSMP.txt"
# Our Knowledge of the External World as a Field for Scientiﬁc Method in Philosophy

book1 = open(file1, encoding='UTF-8').read()
book2 = open(file2, encoding='UTF-8').read()
book3 = open(file3, encoding='UTF-8').read()
book4 = open(file4, encoding='UTF-8').read()

# lowercase
book1 = book1.lower() 
book2 = book2.lower()
book3 = book3.lower()
book4 = book4.lower()

# making a corpus of 5 books
corpus = book1 + book2 + book3 + book4

ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0,1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input.2 

2A smarter way is to parse the whole corpus to ﬁgure out how many distinct characters you have in the corpus (the number may be less than 256, e.g., 53). One can also disregard lowercase and uppercase letters or even remove punctuation characters such as !. 

In [2]:
# figuring out distinct characters
chc = sorted(list(set(corpus)))
ch_lst = enumerate(chc)

print(list(ch_lst))

[(0, '\n'), (1, ' '), (2, '!'), (3, '"'), (4, '&'), (5, "'"), (6, '('), (7, ')'), (8, '*'), (9, '+'), (10, ','), (11, '-'), (12, '.'), (13, '/'), (14, '0'), (15, '1'), (16, '2'), (17, '3'), (18, '4'), (19, '5'), (20, '6'), (21, '7'), (22, '8'), (23, '9'), (24, ':'), (25, ';'), (26, '='), (27, '>'), (28, '?'), (29, '['), (30, ']'), (31, '_'), (32, 'a'), (33, 'b'), (34, 'c'), (35, 'd'), (36, 'e'), (37, 'f'), (38, 'g'), (39, 'h'), (40, 'i'), (41, 'j'), (42, 'k'), (43, 'l'), (44, 'm'), (45, 'n'), (46, 'o'), (47, 'p'), (48, 'q'), (49, 'r'), (50, 's'), (51, 't'), (52, 'u'), (53, 'v'), (54, 'w'), (55, 'x'), (56, 'y'), (57, 'z'), (58, '{'), (59, '|'), (60, '}'), (61, '\xa0'), (62, '§'), (63, '·'), (64, 'â'), (65, 'æ'), (66, 'è'), (67, 'é'), (68, 'ë'), (69, 'î'), (70, 'ï'), (71, 'ô'), (72, 'ö'), (73, 'ü'), (74, 'œ'), (75, 'ŭ'), (76, 'α'), (77, 'β'), (78, 'γ'), (79, 'η'), (80, 'θ'), (81, 'ι'), (82, 'κ'), (83, 'λ'), (84, 'ν'), (85, 'ο'), (86, 'π'), (87, 'ρ'), (88, 'σ'), (89, 'τ'), (90, 'φ'), (91,

In [3]:
# removing irrelevant charaters

for i in range(len(chc)):
    if i == 1 or i==10 or i==12 or (31<i<58): # space, ',', '.', alphabet 
        continue
    else:
        corpus = corpus.replace(chc[i],"")

In [4]:
chc = sorted(list(set(corpus)))

lu_lst =list(enumerate(chc))
lu_dic = dict()  

# making lookup table
for i in range(len(lu_lst)):
    lu_dic[lu_lst[i][1]]=lu_lst[i][0]
print(lu_dic)

{' ': 0, ',': 1, '.': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}


iii. Choose a window size, e.g., W = 100. 

iv. Inputs to the network will be the ﬁrst W−1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence. Basically, we are training the network to predict each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text. For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The ﬁrst input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc. 

In [5]:
W = 100  # window
N = len(corpus)  # the number of charaters
X = []
y = []
for i in range(N-(W-1)):
    seq_X = corpus[i:i + W-1] # 99 characters of each seq.
    seq_y = corpus[i + W -1] # 100th character
    # encoding each character into an integer by lu_table
    X.append([lu_dic[char] for char in seq_X])
    y.append(lu_dic[seq_y])

In [6]:
import numpy as np

X_train = np.reshape(X, (len(X), 99, 1))
# rescaling: dividing by max value, 28
X_train = X_train /28

v. Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

N=29

In [7]:
from keras.utils import np_utils
# one hot encode the output variable
y_train = np_utils.to_categorical(y)

Using TensorFlow backend.


In [8]:
X_train.shape

(1534336, 99, 1)

In [9]:
y_train.shape

(1534336, 29)

vi. Use a single hidden layer for the LSTM with N = 256 (or less) memory units. (N=29)

vii. Use a Softmax output layer to yield a probability prediction for each of the characters between 0 and 1. This is actually a character classiﬁcation problem with N classes. Choose log loss (cross entropy) as the objective function for the network (research what it means).3 

3In Keras, you can use the ADAM optimization algorithm for speed.

In [10]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# LSTM model
lstm = Sequential()
lstm.add(LSTM(29, input_shape=(99, 1)))
lstm.add(Dense(29, activation='softmax'))
lstm.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.


x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss. 

In [11]:
from keras.callbacks import ModelCheckpoint
# checkpointing
file="improved_weights-{epoch:02d}-{loss:.4f}.hdf5"
ch_pt=ModelCheckpoint(file, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks=[ch_pt]

viii. We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text. 

ix. Choose a reasonable number of epochs4 for training, considering your computational power (e.g., 30, although the network will need more epochs to yield a better model).

4one epoch = one forward pass and one backward pass of all the training examples. 

batch size = the number of training examples in one forward/backward pass. The higher the batch size, the more memory space you’ll need. 

number of iterations = number of passes, each pass using [batch size] number of examples. To be clear, one pass = one forward pass + one backward pass (we do not count the forward pass and backward pass as two diﬀerent passes). See https://stats.stackexchange.com/questions/153531/what-is-batch-size-in-neural-network

In [12]:
lstm.fit(X_train, y_train, epochs=10, batch_size=128, callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1534336/1534336 [==============================] - 609s 397us/step - loss: 2.7296

Epoch 00001: loss improved from inf to 2.72961, saving model to improved_weights-01-2.7296.hdf5
Epoch 2/10
1534336/1534336 [==============================] - 601s 392us/step - loss: 2.6030

Epoch 00002: loss improved from 2.72961 to 2.60301, saving model to improved_weights-02-2.6030.hdf5
Epoch 3/10
1534336/1534336 [==============================] - 650s 424us/step - loss: 2.5425

Epoch 00003: loss improved from 2.60301 to 2.54249, saving model to improved_weights-03-2.5425.hdf5
Epoch 4/10
1534336/1534336 [==============================] - 851s 555us/step - loss: 2.4945s - ETA

Epoch 00004: loss improved from 2.54249 to 2.49454, saving model to improved_weights-04-2.4945.hdf5
Epoch 5/10
1534336/1534336 [==============================] - 1060s 691us/step - loss: 2.4554

Epoch 00005: loss improved from 2.49454 to 2.45537, saving model to improved_w

xi. Use the network with the best weights to generate 1000 characters.

In [13]:
# load the file with the smallest loss value
file = "improved_weights-10-2.3494.hdf5"
lstm.load_weights(file)
lstm.compile(loss='categorical_crossentropy', optimizer='adam') 

In [14]:
seed = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
seed=seed.lower()
print(seed)

there are those who take mental phenomena naively, just as they would physical phenomena. this school of psychologists tends not to emphasize the object.


In [15]:
# input sequence (99 characters) from the seed
seq = []
for char in seed:
    seq.append(lu_dic[char])
in_seq = seq[-99:]  # 99 characters
print(in_seq)

[22, 0, 3, 21, 0, 22, 10, 7, 27, 0, 25, 17, 23, 14, 6, 0, 18, 10, 27, 21, 11, 5, 3, 14, 0, 18, 10, 7, 16, 17, 15, 7, 16, 3, 2, 0, 22, 10, 11, 21, 0, 21, 5, 10, 17, 17, 14, 0, 17, 8, 0, 18, 21, 27, 5, 10, 17, 14, 17, 9, 11, 21, 22, 21, 0, 22, 7, 16, 6, 21, 0, 16, 17, 22, 0, 22, 17, 0, 7, 15, 18, 10, 3, 21, 11, 28, 7, 0, 22, 10, 7, 0, 17, 4, 12, 7, 5, 22, 2]


In [16]:
# making reverse lookup table
re_dic = dict()
for i in range(len(lu_lst)):
    re_dic[lu_lst[i][0]]=lu_lst[i][1]
print(re_dic)

{0: ' ', 1: ',', 2: '.', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


In [17]:
import sys

print(seed)
# generating 1000 characters
for i in range(1000):
    X_test = np.reshape(in_seq, (1, 99, 1))
    X_test = X_test / 28
    pred = lstm.predict(X_test, verbose=0)
    out = np.argmax(pred) # output
    ch = re_dic[out]  # converting output to character
    
    sys.stdout.write(ch)
    in_seq.append(out) # adding output to input sequence
    in_seq = in_seq[-99:] # selecting 99 characters

there are those who take mental phenomena naively, just as they would physical phenomena. this school of psychologists tends not to emphasize the object.
 the seale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the teale the t